# Randomized Benchmarking with Gate-dependent Noise
#### Yihong Zhang



## Introduction

This jupyter notebook provides a comparision between Wallman (arXiv: 1703.09835) and Merkel's (arXiv: 1804.05951) paper.

## Pre-setting

#### Superoperator representation

In [256]:
# -*- coding: utf-8 -*-
"""
author: YihongZhang
"""
%pylab inline
%matplotlib inline
import numpy as np
import qutip as qt
import scipy as sp
import random as Random
import heapq as hp

Populating the interactive namespace from numpy and matplotlib


In [128]:
def channel_to_superrep(element, rep_space):
    temp = np.zeros((len(rep_space), len(rep_space)))
    for i in range(len(rep_space)):
        for j in range(len(rep_space)):
            temp[i, j] = (rep_space[i].dag() * element * rep_space[j] * element.dag()).tr()
    return temp

#### 1Q-clifford generation

In [129]:
cliff1Q_generator = qt.qubit_clifford_group(1,0)
clifford1Q = []
normalized_pauli = [qt.identity(2), qt.sigmax(), qt.sigmay(), qt.sigmaz()] / np.sqrt(2)
for i in range(24):
    x = channel_to_superrep(cliff1Q_generator.__next__(), normalized_pauli)
    clifford1Q.append(x)
print('The number of 1Q-clifford elements is:', len(clifford1Q))

The number of 1Q-clifford elements is: 24


## Fourier transform of Clifford group

#### 1Q-clifford fourier transform -- $\tilde{\phi}(\sigma)$

In [275]:
def cliff1Q_FT(Cliff, rep):
    FTrep_num = 4 * len(rep[0])
    temp = np.zeros((FTrep_num, FTrep_num))
    for i in range(24):
        temp = temp + np.kron(Cliff[i], rep[i].conjugate())
    return temp / 24
np.set_printoptions(precision=1)
print('The fourier transform in superoperator rep is: \n', cliff1Q_FT(clifford1Q, clifford1Q))

The fourier transform in superoperator rep is: 
 [[1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.3 0.  0.  0.  0.  0.3 0.  0.  0.  0.  0.3]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.3 0.  0.  0.  0.  0.3 0.  0.  0.  0.  0.3]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0. 

#### 1Q-clifford fourier transform -- $\tilde{\phi}(\sigma_I)$

In [276]:
cliff1Q_generator = qt.qubit_clifford_group(1,0)
cliff1Q_irrepI = []
normalized_identity = [qt.identity(2)] / np.sqrt(2)
for i in range(24):
    x = channel_to_superrep(cliff1Q_generator.__next__(), normalized_identity)
    cliff1Q_irrepI.append(x)    
print('The fourier transform in irrep 1 is: \n', cliff1Q_FT(clifford1Q, cliff1Q_irrepI))

The fourier transform in irrep 1 is: 
 [[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


#### 1Q-clifford fourier transform -- $\tilde{\phi}(\sigma_P)$

In [277]:
cliff1Q_generator = qt.qubit_clifford_group(1,0)
cliff1Q_irrepP = []
normalized_noidentity = [qt.sigmax(), qt.sigmay(), qt.sigmaz()] / np.sqrt(2)
for i in range(24):
    x = channel_to_superrep(cliff1Q_generator.__next__(), normalized_noidentity)
    cliff1Q_irrepP.append(x)
print('The fourier transform in irrep 2 is: \n', cliff1Q_FT(clifford1Q, cliff1Q_irrepP))

The fourier transform in irrep 2 is: 
 [[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.3 0.  0.  0.  0.3 0.  0.  0.  0.3]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.3 0.  0.  0.  0.3 0.  0.  0.  0.3]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.3 0.  0.  0.  0.3 0.  0.  0.  0.3]]


Clearly we can see that  $\tilde{\phi}(\sigma) = \tilde{\phi}(\sigma_I) \oplus \tilde{\phi}(\sigma_P)$, here $\sigma = \sigma_I \oplus \sigma_P$.

## Comparision between Merkel and Wallman's paper

### Mathmatical analysis
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$
We will first make brief theoretical analysis of Merkel and Wallman's paper.
***
##### Merkel
In Merkel's paper, the average fidelity is $1 - \delta$, i.e.,
\begin{equation}
\begin{split}
1 - \delta &= \mathbb{E}_{g\in\mathcal{G}} F_e[\phi(g), \phi_{ideal}(g)] = \sum_{\sigma}\frac{d_\sigma}{d_\phi} Tr[\tilde{\phi}(\sigma) \tilde{\phi}_{ideal}^+(\sigma)] \\
&= \frac{1}{4^n}\bra{1 \oplus \mathbf{0}}\tilde{\phi}(\sigma_I)\ket{1 \oplus \mathbf{0}} + \frac{4^n - 1}{4^n} \bra{\mathbf{0} \oplus \psi}\tilde{\phi}(\sigma_I)\ket{\mathbf{0} \oplus \psi}
\end{split}
\end{equation}
here $\psi$ is the maximally entangled states. Merkel defines
\begin{equation}
\begin{split}
t &\equiv \bra{1 \oplus \mathbf{0}}\tilde{\phi}(\sigma_I)\ket{1 \oplus \mathbf{0}} \\
p &\equiv \bra{\mathbf{0} \oplus \psi}\tilde{\phi}(\sigma_I)\ket{\mathbf{0} \oplus \psi} 
\end{split}
\end{equation}
We can see that $t$ and $p$ are the (largest) eigenvalues of $\tilde{\phi}(\sigma)$. <br>
The average survival probability is
\begin{equation}
S_m = Ap^m + Bt^m + C + \mathcal{O}(\delta^n)
\end{equation}
***
##### Wallman
In Wallman's paper, for any unitary 2-design $\mathcal{G}$ and channel $\mathcal{C}$, the twired channel is
\begin{equation}
\mathbb{E}_{g\in\mathcal{G}} [\phi_{ideal}(g^+) \mathcal{C} \phi_{ideal}(g)] = \frac{t}{d}I_d + p(\rho - \frac{1}{d}I_d) \equiv \mathcal{D}_{p,t}
\end{equation}
here $\phi(g)$ is the super-operator representation; $t$ and $p$ are defined by the eigenvalue equations,
\begin{equation}
\begin{split}
&t:  
\left\{
\begin{aligned}
& \mathbb{E}_{g\in\mathcal{G}}\phi(g)\ket{L}\rangle = t\ket{L}\rangle \\
& \langle\bra{R}\mathbb{E}_{g\in\mathcal{G}}\phi(g) = t\langle\bra{R} 
\end{aligned}
\right. \\
&p:
\left\{
\begin{aligned}
& \mathbb{E}_{g\in\mathcal{G}} [\phi_{ideal}(g) \otimes \phi(g)] \mathrm{vec}(\mathcal{L'}) = p\mathrm{vec}(\mathcal{L'}) \\
& \mathbb{E}_{g\in\mathcal{G}} [\phi(g) \otimes \phi_{ideal}(g)]^T \mathrm{vec}(\mathcal{R'}) = p\mathrm{vec}(\mathcal{R'})
\end{aligned}
\right. \\
\end{split}
\end{equation}
here $t$ and $p$ are the largest eigenvalues. <br>
The average survival probability is
\begin{equation}
S_m = Ap^m + Bt^m + \epsilon_m
\end{equation}
here $|\epsilon| < \delta_1\delta_2^m$, and $\delta_1$ and $\delta_2$ quantify the amount of gate dependence.
***
##### Comparision
Recall the definition of fourier transform of $\phi$ on $\sigma$
\begin{equation}
\tilde{\phi}(\sigma) = \mathbb{E}_{g\in\mathcal{G}} \phi(g) \otimes \sigma^*(g)
\end{equation}
As for super-operator representation, $\sigma^*(g) = \sigma(g)$, we can see that in Wallman's paper, 
\begin{equation}
\begin{split}
&t_{Wallman}:
\left\{
\begin{aligned}
& \tilde{\phi}(\sigma_I)\ket{L}\rangle = t_{Wallman}\ket{L}\rangle \\
& \langle\bra{R}\tilde{\phi}(\sigma_I) = t_{Wallman}\langle\bra{R} 
\end{aligned}
\right. \\
&p_{Wallman}: \mathrm{vec}(\mathcal{R'})^T \tilde{\phi}(\sigma) = p_{Wallman}\mathrm{vec}(\mathcal{R'})^T
\end{split}
\end{equation}
which means $t_{Wallman}$ are the eigenvalue of $\tilde{\phi}(\sigma_I)$ and $p_{Wallman}$ are the eigenvalur of $\tilde{\phi}(\sigma)$. Since $\tilde{\phi}(\sigma) = \tilde{\phi}(\sigma_I) \oplus \tilde{\phi}(\sigma_P)$, $t_{Wallman}$ and $p_{Wallman}$ are the eigenvalues of $\tilde{\phi}(\sigma)$. <br>
In conclusion,
\begin{equation}
\begin{split}
t_{Merkel} = t_{Wallman} \\
p_{Merkel} = p_{Wallman}
\end{split}
\end{equation}

### Numerical simulation

Here we use the generalized depolarizing channel, i.e.,
\begin{equation}
\begin{split}
\varepsilon_{dep}(\rho) &= \frac{t}{d}I_d + p(\rho - \frac{1}{d}I_d) \\
&= (p + \frac{t - p}{2d})\rho + \frac{t - p}{2d}(X\rho X + Y\rho Y + Z\rho Z)
\end{split}
\end{equation}

In [278]:
rz_angle = 0.09 #phase error parameter (radian measure)
t = 1 #depolarizing channel parameters
p = 0.99
#phase error (z rotation)
phase_err = channel_to_superrep(qt.rz(rz_angle), normalized_pauli)

#depolarizing channel in super-operator rep
def dep(t, p, num_qubit, rep_space):
    temp = np.zeros((len(rep_space), len(rep_space)))
    d = 2**num_qubit
    for i in range(len(rep_space)):
        for j in range(len(rep_space)):
            temp_chan = (p + (t - p)/2/d) * rep_space[j] + (t - p)/2/d * (qt.sigmax() * rep_space[j] * qt.sigmax() + qt.sigmay() * \
                                                                        rep_space[j] * qt.sigmay() + qt.sigmaz() * rep_space[j] * qt.sigmaz())
            temp[i, j] = (rep_space[i].dag() * temp_chan).tr()
    return temp
dep_chan = dep(t, p, 1, normalized_pauli)

np.set_printoptions(precision=3)
print('The phase error Rz(',rz_angle ,'):\n',phase_err)
print('The depolarizing channel D(',t,',',p,'):\n', dep_chan)

The phase error Rz( 0.09 ):
 [[ 1.     0.     0.     0.   ]
 [ 0.     0.996 -0.09   0.   ]
 [ 0.     0.09   0.996  0.   ]
 [ 0.     0.     0.     1.   ]]
The depolarizing channel D( 1 , 0.99 ):
 [[1.   0.   0.   0.  ]
 [0.   0.99 0.   0.  ]
 [0.   0.   0.99 0.  ]
 [0.   0.   0.   0.99]]


We use Wallman's error model to construct noisy clifford process. All Cliffords have a uniform depolarizing error $D(t, p)$, but half of them additionally have a small rotation about Z (we choose 12 elements randomly from the whole 24 elements\). 

In [279]:
Cliff1QIdeal = clifford1Q
np.set_printoptions(precision=3)
Cliff1QNoisy = []
for i in range(len(Cliff1QIdeal)):
    Cliff1QNoisy.append(np.dot(dep_chan, Cliff1QIdeal[i]))
#np.set_printoptions(precision=5)
#print(Cliff1QNoisy)
NoisySequence = Random.sample(range(24), 12)
for i in range(len(NoisySequence)):
    Cliff1QNoisy[NoisySequence[i]] = np.dot(phase_err, Cliff1QNoisy[NoisySequence[i]])

def EntangleFedelity(channel1, channel2):
    temp = 0
    for i in range(len(channel1)):
        temp = temp + (np.dot(channel1[i], (channel2[i].T).conjugate())).trace() / 4
    return temp / 24
print('The entanglement fidelity is:', EntangleFedelity(Cliff1QNoisy, Cliff1QIdeal))

The entanglement fidelity is: 0.9914983014204682


##### Fidelity calculated by Merkel's analysis

In [291]:
FourierNoisy_irrepI = cliff1Q_FT(Cliff1QNoisy, cliff1Q_irrepI)
FourierNoisy_irrepP = cliff1Q_FT(Cliff1QNoisy, cliff1Q_irrepP)
eigenvalNoisy_irrepI, eigenvecNoisy_irrepI = np.linalg.eig(FourierNoisy_irrepI)
eigenvalNoisy_irrepP, eigenvecNoisy_irrepP = np.linalg.eig(FourierNoisy_irrepP)
LargestEigenval_irrepI = hp.nlargest(1, eigenvalNoisy_irrepI)
LargestEigenval_irrepP = hp.nlargest(1, eigenvalNoisy_irrepP)
FourierFidelity = LargestEigenval_irrepI[0] / 4 + LargestEigenval_irrepP[0] * 3 / 4
print('The largest eigenvalue in irrepI is:', LargestEigenval_irrepI)
print('The largest eigenvalue in irrepP is:', LargestEigenval_irrepP)
print('The entanglement fidelity calculated by fourier analysis is:', FourierFidelity)

The largest eigenvalue in irrepI is: [0.9999999999999997]
The largest eigenvalue in irrepP is: [(0.9885539234147862+0j)]
The entanglement fidelity calculated by fourier analysis is: (0.9914154425610896+0j)


#### Fidelity calculated by Wallman's analysis

In [294]:
FourierNoisy = cliff1Q_FT(Cliff1QNoisy, clifford1Q)
eigenvalNoisy, eigenvecNoisy = np.linalg.eig(FourierNoisy)
LargestEigenval = hp.nlargest(2, eigenvalNoisy)
#print('The fourier transform of noisy Cifford is: \n', FourierNoisy)
print('The eigenvalues of noisy fourier tranform are: \n',eigenvalNoisy)
print('The two largest eigenvalues of noisy fourier tranform are: \n', LargestEigenval)
print('t in Wallmans paper is:', LargestEigenval_irrepI[0])
print('p in Wallmans paper is:', LargestEigenval[0])

The eigenvalues of noisy fourier tranform are: 
 [ 0.000e+00+0.000e+00j  4.626e-18+4.626e-18j  4.626e-18-4.626e-18j
  9.886e-01+0.000e+00j  6.977e-03+0.000e+00j -1.403e-03+5.828e-03j
 -1.403e-03-5.828e-03j -3.793e-03+0.000e+00j  6.671e-05+0.000e+00j
 -3.374e-03+0.000e+00j -2.749e-10+0.000e+00j  2.749e-10+0.000e+00j
  2.528e-18+0.000e+00j -2.257e-17+0.000e+00j -3.414e-16+0.000e+00j
  1.000e+00+0.000e+00j]
The two largest eigenvalues of noisy fourier tranform are: 
 [(0.9999999999999997+0j), (0.9885539234147851+0j)]
t in Wallmans paper is: 0.9999999999999997
p in Wallmans paper is: (0.9999999999999997+0j)
